## Exploration of Data

In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import scipy.io as sio
import os

# Load image data
data_load_1 = sio.loadmat('Proj2FeatVecsSet1.mat')
data_load_2 = sio.loadmat('Proj2TargetOutputsSet1.mat')
data_set = data_load_1['Proj2FeatVecsSet1']
data_target = data_load_2['Proj2TargetOutputsSet1']

In [2]:
# Divide my target data into nice 1-D classifier
number_labels = []
for ars in data_target:
    if np.all(ars == [1,-1,-1,-1,-1]):
        ars = 1
        number_labels.append(ars)
    elif np.all(ars == [-1,1,-1,-1,-1]):
        ars = 2
        number_labels.append(ars)
    elif np.all(ars == [-1,-1,1,-1,-1]):
        ars = 3
        number_labels.append(ars)
    elif np.all(ars == [-1,-1,-1,1,-1]):
        ars = 4
        number_labels.append(ars)
    elif np.all(ars == [-1,-1,-1,-1,1]):
        ars = 5
        number_labels.append(ars)
        
number_labels = np.asarray(number_labels)

## Define how many components we should use and run PCA

In [3]:
from sklearn.decomposition import PCA
pca = PCA().fit(data_set)
cum_var = np.cumsum(pca.explained_variance_ratio_)
eigenvalues = pca.explained_variance_

count = 0
for var in cum_var:
    count += 1
    if var >= 0.95:
        n_components = count
#         answer = "We need about "+ str(n_components) + " components to retain 95% of the variance"
#         print(answer)
        break
        
# plt.figure(1)
# plt.plot(cum_var)
# plt.xlabel('Number of Components')
# plt.ylabel('Cumulative Explained Variance')
# plt.figure(2)
# plt.plot(eigenvalues)
# plt.xlabel('Number of Components')
# plt.ylabel('Eigenvalues')
# plt.show()

# Minumum Noise Factor --> Similar to PCA but removes noise from bands

In [4]:
from sklearn.decomposition import PCA
from skimage.transform import rescale
from sklearn.cluster import KMeans
import numpy as np
import time

#Using PCA
pca = PCA(n_components=n_components)
reduced_data = pca.fit_transform(data_set)

## Creation of Data Folds - Training and Testing

In [5]:
from sklearn.model_selection import train_test_split
# #############################################################################
# Split into a training set and a test set using a stratified k fold
X_train, X_test, y_train, y_test = train_test_split(
    reduced_data, number_labels, test_size=0.20, stratify = number_labels)

## Data breakdown - For X_train_1 and y_train_1, take sample of 1000 out of 5000

In [6]:
# This can be done nicely with the kfold function
X_train_1 = X_train[:1000]
X_train_2 = X_train[5000:9999]
X_train_3 = X_train[10000:14999]
X_train_4 = X_train[15000:20000]
y_train_1 = y_train[:1000]
y_train_2 = y_train[5000:9999]
y_train_3 = y_train[10000:14999]
y_train_4 = y_train[15000:20000]

## TrainMyClassifierParameters Function

In [16]:
def TrainMyClassifierParameters(Algorithm):
    if Algorithm == 'SVM':
        Parameters = {
            'C' : 0.1,
            'gamma' : 0.1
        }
    elif Algorithm == 'RVM':
        Parameters = {
            'alpha' : 1e-06,
            'beta' : 1e-06
        }
    elif Algorithm == 'GP':
        Parameters = {
            'length_scale' : 10             
        }
    return Parameters, Algorithm

## TrainMyClassifier Function

In [160]:
# Not sure if this is the correct type of function we need
def TrainMyClassifier(XEstimate, YEstimate, XValidate, TrainMyClassifierParameters):
    from sklearn.svm import SVC
    from skrvm import RVC
    from sklearn.gaussian_process import GaussianProcessClassifier
    from sklearn.multiclass import OneVsOneClassifier
    from sklearn.model_selection import GridSearchCV
    from sklearn.gaussian_process.kernels import RBF
    from time import time
    t0 = time()
    # Paramaters should have this shape in order for it to work ==>  Parameters = {'C': [1e3, 1e4, 1e5], 'gamma': [0.001, 0.01, 0.1] }
    if TrainMyClassifierParameters[1] == 'SVM':
        # ################################################
        # Train a SVM classification model
        print("Fitting the SVM classifier to the training set")
        param_grid = TrainMyClassifierParameters[0]
        clf = SVC(kernel='rbf', class_weight='balanced', decision_function_shape = 'ovo',**TrainMyClassifierParameters[0])
        clf = clf.fit(XEstimate, YEstimate)
        params = clf
        y_pred = clf.predict(XValidate)
        scores = clf.score(XEstimate,YEstimate)
        print("done in %0.3fs" % (time() - t0))
        return y_pred, scores, params
    elif TrainMyClassifierParameters[1] == 'RVM':
        # #############################################################################
        # Train a RVM classification model
        print("Fitting the RVM classifier to the training set")
        t0 = time()
        clf = RVC(kernel='rbf',verbose=True, n_iter=1,**TrainMyClassifierParameters[0])
        clf.fit(XEstimate, YEstimate)
        params = clf
        y_pred = clf.predict(XValidate)
        scores = clf.score(XEstimate,YEstimate)
        print("done in %0.3fs" % (time() - t0))
        return y_pred, scores, params
    elif TrainMyClassifierParameters[1] == 'GP':
       # #############################################################################
        # Train a GP classification model
        print("Fitting the GP classifier to the training set")
        t0 = time()
        k_rbf = 1 * RBF(length_scale=TrainMyClassifierParameters[0]['length_scale'])
        clf = GaussianProcessClassifier(kernel = k_rbf, multi_class = 'one_vs_one')
        clf.fit(X_train_1, y_train_1)
        params = clf
        y_pred = clf.predict(XValidate)
        scores = clf.score(XEstimate,YEstimate)
        print("done in %0.3fs" % (time() - t0))
        return y_pred, scores, params
    else:
        print("Incorrect type of algorithm, please use only one of the supported classifiers SVM, RVM, GP")

## MyConfusionMatrix Function 

In [161]:
def MyConfusionMatrix(Y,YValidate,ClassNames):
    from sklearn.metrics import classification_report
    from sklearn.metrics import confusion_matrix
    from sklearn.metrics import accuracy_score
    import pandas as pd
    c_r = classification_report(YValidate, Y)
    c_m = confusion_matrix(YValidate, Y)
    c_m = np.round((c_m / c_m.astype(np.float).sum(axis=0)), 3)
    a_s = accuracy_score(YValidate, Y)
    
    # labels = ['One','Two','Three','Four','Five'] - This is the format of the labels
    labels = ClassNames
    df = pd.DataFrame(c_m, dtype='str', index=labels)
    df.columns = ClassNames
    return c_m, df, a_s

## MyCrossValidate Function

In [143]:
def MyCrossValidate(XTrain,YTrain2,Nf,Algorithm): #Why do we use a YTrain with '2' index?
    from sklearn.model_selection import KFold
    dict = {}   
    pca = PCA(n_components=9)
    reduced_data = pca.fit_transform(XTrain)
    #print reduced_data.shape
    kf = KFold(n_splits=Nf)
    kf.get_n_splits(XTrain)
    EstParameters = []
    EstConfMatrices = []
    ConfMatrix=np.zeros((5,5),dtype=int)
    YTrain = []
    i=0        
    for train_index, test_index in kf.split(XTrain):
        # down-sampling
        XEst1 = XTrain[train_index]
        YEst1 = YTrain2[train_index]      
        XValid = XTrain[test_index]
        YValid = YTrain2[test_index]  
        XEst = XEst1[:2000]
#         XEst = np.concatenate((XEst,XEst1[4000:6000,]))
#         XEst = np.concatenate((XEst,XEst1[8000:10000,]))
#         XEst = np.concatenate((XEst,XEst1[12000:14000,]))
        print XEst.shape
        
        YEst = YEst1[:2000]
#         YEst = np.append(YEst,YEst1[4000:6000])
#         YEst = np.append(YEst,YEst1[8000:10000])
#         YEst = np.append(YEst,YEst1[12000:14000])
        
        TrainMyClassifierParameters = []
        TrainMyClassifierParameters.append({'alpha' : 1e-04,'beta' : 1e-04 })
        TrainMyClassifierParameters.append(Algorithm)
        y_pred1, scores1, params1 = TrainMyClassifier(XEst,YEst,XValid,TrainMyClassifierParameters)
        y_pred = y_pred1
        EstParameters.append(params1)
        dict[i]= {'scores':[scores1],'alpha' : 1e-04,'beta' : 1e-04}
#         TrainMyClassifierParameters[0] = {'alpha' : 1e-01,'beta' : 1e-04 }
#         y_pred2, scores2, params2 = TrainMyClassifier(XEst,YEst,XValid,TrainMyClassifierParameters)
#         TrainMyClassifierParameters[0] = {'alpha' : 5e-02,'beta' : 5e-02 }
#         y_pred3, scores3, params3 = TrainMyClassifier(XEst,YEst,XValid,TrainMyClassifierParameters)
#         if max(scores1,scores2,scores3)==scores1:
#             y_pred = y_pred1
#             EstParameters.append(params1)
#             dict[i]= {'scores':[scores1,scores2,scores3],'alpha' : 1e-02,'beta' : 1e-02}
#         elif max(scores1,scores2,scores3)==scores2:
#             y_pred = y_pred2
#             EstParameters.append(params2)
#             dict[i]= {'scores':[scores1,scores2,scores3],'alpha' : 1e-01,'beta' : 1e-01}
#         elif max(scores1,scores2,scores3)==scores3:
#             y_pred = y_pred3  
#             EstParameters.append(params3)
#             dict[i]= {'scores':[scores1,scores2,scores3],'alpha' : 5e-02,'beta' : 5e-02}
        c_m, df, a_s = MyConfusionMatrix(y_pred, YValid,['One','Two','Three','Four','Five'])
        
        ConfMatrix=ConfMatrix+c_m
        EstConfMatrices.append(df)
        
        YTrain.append(y_pred)
        i=i+1
    np.save(Algorithm+'.npy',dict)
    ConfMatrix = ConfMatrix/5 
    print(ConfMatrix.astype(np.float).sum(axis=0))
    return YTrain, EstParameters, EstConfMatrices, ConfMatrix

## TestMyClassifier Function

In [144]:
def TestMyClassifier(XTest, Parameters, EstParameters):
    # Do similar to trainmyclassifer but with the data from MyCrossValidation
    from sklearn.metrics import classification_report
    from sklearn.metrics import confusion_matrix
    print("Predicting class on the test set")
    t0 = time()
#     pred_svc=SVC()
#     pred_svc.support_ = EstParameters[0]
#     pred_svc.support_vectors_ = EstParameters[1]
#     pred_svc.n_support_ = EstParameters[2]
#     pred_svc.dual_coef_ = EstParameters[3]
#     pred_svc.intercept_ = EstParameters[4]
#     pred_svc.probA_ = EstParameters[5]
#     pred_svc.probB_ = EstParameters[6]
#     pred_svc.fit_status_ = EstParameters[7]
#     pred_svc._sparse = EstParameters[8]
#     pred_svc.shape_fit_ = EstParameters[9]
#     pred_svc._dual_coef_ = EstParameters[10]
#     pred_svc._intercept_ = EstParameters[11]
#     pred_svc._gamma = EstParameters[12]
#     pred_svc.classes_ = EstParameters[13]
  
#     pred_svc.set_params(**EstParameters[14])
    pred_svc = EstParameters
    Ytest = pred_svc.predict(XTest)
    print("done in %0.3fs" % (time() - t0))
#     print(Ytest)
#     print(classification_report(y_test, Ytest))
#     print(confusion_matrix(y_test, Ytest))
    return YTest

In [135]:
YTrain,EstParameters,EstConfMatrices,ConfMatrix = MyCrossValidate(X_train,y_train,5,'RVM')
print EstParameters
print EstConfMatrices
print ConfMatrix

(2000, 9)
Fitting the RVM classifier to the training set
done in 12.228s
(2000, 9)
Fitting the RVM classifier to the training set
done in 12.595s
(2000, 9)
Fitting the RVM classifier to the training set
done in 12.088s
(2000, 9)
Fitting the RVM classifier to the training set
done in 12.210s
(2000, 9)
Fitting the RVM classifier to the training set
done in 12.384s
[ 1.0002  1.0002  1.0002  1.0002  1.0002]


AttributeError: 'list' object has no attribute 'support_vectors_'

In [165]:
print ConfMatrix
print EstParameters[0].relevance_.shape[0]

[[ 0.953   0.0086  0.011   0.0026  0.0058]
 [ 0.0104  0.8934  0.0054  0.097   0.0012]
 [ 0.013   0.004   0.9424  0.0086  0.0284]
 [ 0.0106  0.092   0.0108  0.862   0.0248]
 [ 0.0132  0.0022  0.0306  0.03    0.94  ]]


AttributeError: 'RVC' object has no attribute 'relevance_'

In [137]:
read_dictionary = np.load('RVM.npy').item()
print read_dictionary

{0: {'alpha': 0.0001, 'beta': 0.0001, 'scores': [0.9365]}, 1: {'alpha': 0.0001, 'beta': 0.0001, 'scores': [0.93600000000000005]}, 2: {'alpha': 0.0001, 'beta': 0.0001, 'scores': [0.93600000000000005]}, 3: {'alpha': 0.0001, 'beta': 0.0001, 'scores': [0.93600000000000005]}, 4: {'alpha': 0.0001, 'beta': 0.0001, 'scores': [0.93600000000000005]}}


In [162]:
YTrain1,EstParameters1,EstConfMatrices1,ConfMatrix1 = MyCrossValidate(X_train,y_train,5,'RVM')
print EstParameters1[0]
print EstConfMatrices1
print ConfMatrix1

(2000, 9)
Fitting the RVM classifier to the training set
Iteration: 0
Alpha: [  3.18338877e-02   2.16129055e-01   1.17716975e-02   6.14913025e-03
   3.21295345e-02   4.76900318e-01   7.80743195e+00   5.77246373e-03
   4.40803676e-02   6.12672141e-02   3.87549092e+00   3.37384755e-02
   1.44341024e-02   1.05377856e-02   5.10997473e-02   8.23837587e-02
   2.83678285e-02   2.94661585e-02   1.18617687e-02   1.10345019e-02
   3.06425407e-02   4.78134319e-03   2.02919518e-01   1.41517608e-02
   2.36729514e-02   4.54159366e-02   2.51161573e-02   6.21397185e+00
   9.93490428e-02   7.21583320e-02   1.39546206e-02   2.42641188e-02
   5.23510536e-02   1.70378869e-02   1.01325507e-02   5.93287580e-02
   1.65575734e-02   1.14240683e-02   2.72141782e-01   8.35510049e-03
   4.22118098e-02   9.61474525e-03   8.28619429e-02   2.22571153e-02
   2.88808700e-02   3.12435440e-02   1.82473212e-02   2.24272628e-02
   4.08197948e-02   5.49398234e-03   2.09960725e-01   3.59832902e-02
   5.66299395e-02   4.6642

Iteration: 0
Alpha: [  1.08339627e-03   2.99480400e-02   2.28386886e-03   2.89718549e-01
   1.71081685e-02   1.69672430e-03   4.38583261e-03   1.95310834e-03
   2.17106727e-03   4.70881462e-04   4.50594405e-02   6.81236838e-03
   5.72425201e-03   1.01403935e-02   1.26936325e-03   2.76879849e-03
   3.91478997e-02   1.27051093e-03   1.12311479e-03   4.10019188e-03
   5.82296937e-03   1.37770195e-03   1.39915599e-03   4.09567245e-03
   3.88493931e-03   2.14097105e-02   1.36627671e-02   4.81015686e-02
   3.78631955e-01   1.68316376e-02   1.08336574e-01   3.52818515e-03
   1.42062688e-02   8.20154313e-03   8.74914262e-03   1.04795481e-02
   7.61738925e-01   1.37584931e-03   6.68645533e-04   9.64391284e-04
   7.56700601e-04   1.31938776e-03   2.96521132e-03   2.27070983e-02
   4.90371326e-03   6.46551462e-01   2.63339660e-03   3.62724589e+00
   5.56529692e-03   2.30150926e-03   5.17269946e-03   5.04589563e-03
   4.11293802e-04   2.54443217e-03   2.24520624e-03   1.52616737e-03
   4.56626705e

Iteration: 0
Alpha: [  4.73444022e-03   3.98712039e-04   3.29827755e-04   9.45668424e-04
   2.73616369e-04   3.47599509e+00   7.19972938e-02   7.75216571e-03
   1.54445780e-03   2.08335318e-04   1.26458346e-03   2.45564007e-03
   1.06722134e-02   6.67194867e-04   2.15594761e-03   2.49550580e-03
   3.23119765e-04   6.44995697e-03   2.79917402e-03   6.12200803e-04
   1.16487182e-03   6.30042865e-03   2.56579032e-02   8.00031239e-03
   4.19336194e-03   7.21608613e-02   1.55143395e-03   1.05668225e-03
   1.63094702e-03   1.00627121e-03   6.24798304e-04   6.04587968e-04
   1.25482536e-02   5.28681880e-03   6.69129026e-02   1.06577545e-03
   1.24279004e-03   7.44540440e-02   2.40453201e-03   2.04070943e-02
   1.46380797e-02   4.71708673e-04   1.94316599e-02   5.74155651e-04
   8.55303981e-03   2.60854262e-03   1.41965935e-03   1.58227739e-02
   5.04575565e-03   1.11664715e-03   1.05434937e-03   6.23618986e-03
   9.56123228e-03   4.02375189e-03   7.58122754e-02   1.19260321e-02
   5.19059703e

Iteration: 0
Alpha: [  3.16087872e-03   2.80992579e-01   2.47622414e-03   9.55894833e-04
   4.45802318e-04   1.05396997e-03   3.84318052e-04   3.55053699e-03
   3.17712132e-02   7.98684529e-04   4.06971352e-03   2.90962097e-03
   6.33567114e-04   8.82311162e-04   4.24563023e+01   4.24551885e-03
   4.51427393e-03   6.75922363e-03   2.93512194e-02   4.46964482e-03
   2.07896645e-03   2.55561464e-02   8.63434820e-04   6.11367268e-03
   1.06489254e-03   1.78092538e-02   4.53168032e-03   5.48585914e-03
   1.16068347e-02   7.71057223e-03   1.58595893e-02   7.51213479e-04
   3.75264524e-04   4.78352480e-02   8.33576363e-03   1.96975464e-03
   1.32014594e-03   3.69283823e-04   2.13823050e-02   2.29953640e-02
   5.62173921e-03   6.88306401e-04   2.88538428e-04   6.40471804e-03
   2.19240274e-03   4.34751810e-03   1.50738637e+01   1.66007422e-03
   3.80325017e-03   1.18169169e-03   9.56648726e-04   2.97161730e-03
   3.93560064e-03   2.91428317e-03   7.51379369e-03   1.68251381e-03
   1.81966305e

Iteration: 0
Alpha: [  2.70667187e-02   1.49762030e-02   3.18956241e-02   7.54796975e-03
   1.46499793e-02   1.32412715e-01   5.01174647e-03   2.18658486e-01
   1.62561568e-02   2.64137093e-01   1.93733421e-02   7.29751056e-03
   1.87832880e-01   1.59101529e-02   4.33510540e-02   1.90086470e+00
   1.65880235e-01   8.72936251e-02   7.36145555e-02   1.34283487e-02
   1.73266476e-01   9.90469635e-03   2.95461525e+01   8.49236970e+00
   2.58128595e-01   8.04554724e+01   5.86688202e-02   1.74935282e-02
   1.88634159e+00   3.20224792e-02   2.23051941e-02   6.53407305e-03
   2.25054769e+00   6.45879130e-01   9.01937180e-03   9.53102147e-03
   1.79332747e-01   5.26572688e-02   4.32684651e-02   8.57448616e-03
   1.59913507e-02   7.46555671e-03   1.17508671e-02   8.54162875e-03
   1.39003856e-02   1.60948494e-01   3.36350554e+00   1.38018795e-02
   5.07755749e-03   1.50634512e-02   1.51309591e-01   1.67632816e-01
   1.91531898e-02   3.16143276e-01   2.29123946e-01   2.21308161e-01
   3.62258669e

...

Iteration: 0
Alpha: [  6.09692572e-04   1.36273732e-01   4.43632429e-02   7.64552094e-04
   6.99991838e-04   3.30942088e-03   1.12730969e-03   2.97104936e-04
   4.45738224e-03   3.70203427e-04   1.12953505e-04   9.21568111e-04
   1.02057330e-02   4.58465761e-04   4.64422884e-04   2.21241441e-02
   1.76765378e-01   1.78372384e-02   1.31006324e-03   3.84444341e-03
   6.83359118e-05   6.20647733e-04   4.72669543e-04   3.11466112e-04
   2.71061124e-04   8.90726101e-04   5.16923327e-04   1.83838558e-03
   1.78652659e-04   4.46035141e-04   3.97226592e-04   7.33795917e-04
   2.42364404e-03   4.15358685e-04   1.02710042e-04   1.18635913e-02
   2.70040334e-04   6.19828816e-04   4.26441336e-04   2.14619770e-04
   1.76314804e-03   5.74494626e-04   3.73664695e-04   2.07288274e-03
   2.73939878e-04   2.47511330e-03   3.16416136e-03   3.57243476e-03
   4.79629900e-03   3.59086175e-04   8.75602841e-04   2.58085086e-04
   3.25825649e-04   8.66463387e-04   1.30337712e-04   2.86876877e-03
   3.87719086e

Iteration: 0
Alpha: [  5.72180424e-03   2.49823937e-02   8.36572858e-02   7.14113209e-02
   2.42083472e-02   4.42495656e-03   1.38425651e-01   7.45696593e-02
   7.09630509e-03   1.70537899e-02   5.06205392e-01   3.97685096e-02
   6.85734212e-03   1.06853110e-02   1.23914773e-01   1.23700251e-01
   9.88674705e-03   2.64407822e-03   1.18894146e-02   4.18462698e-01
   3.65130811e-02   4.46528190e-02   9.90500291e-03   4.67859253e-02
   3.90774467e-02   7.47718535e-03   2.11923442e-02   3.24105264e-02
   5.41219724e-03   2.64485397e-02   2.32080690e-02   2.18630864e-02
   1.02660032e-02   2.03794266e-02   6.17242170e-03   5.92199173e-02
   4.65922553e-02   8.84644763e-03   6.39300690e-03   1.59772439e+00
   1.12414426e-02   1.24893813e-02   1.36530088e-02   1.16817615e-02
   2.95863050e-02   1.05360894e-02   1.95718085e+00   5.96347644e-02
   1.33652460e-02   5.91927992e-03   1.21286889e-02   7.92825602e-03
   3.90052353e-03   2.92833537e-02   2.22439065e-02   1.97273691e-01
   1.24635042e

Iteration: 0
Alpha: [  8.44931230e-02   9.97129739e-01   3.04985519e-01   1.94411652e-02
   1.03569319e+00   3.75561660e-01   6.32177132e-03   2.76932933e-02
   8.32416377e-01   1.19392416e-01   1.87161094e-01   1.00217503e-01
   1.46770838e-02   1.66476881e-01   7.20973204e-03   9.50787731e-02
   2.62303438e-02   3.17139304e-02   6.77078754e-02   5.97588555e-03
   2.31456907e-01   9.37347849e-02   6.63521605e-03   7.44129512e+00
   3.15800594e-02   1.22202226e-02   6.36553799e-02   1.89022267e-01
   9.63757976e-03   1.34824350e-01   1.07927664e-01   2.95638507e+01
   1.44548417e-02   6.55486180e-03   9.09856916e-02   7.23533159e-02
   5.70085258e-03   3.37133605e-01   5.04169071e-02   4.12915806e+00
   4.95699702e-03   1.61220676e-01   3.54000239e-02   2.08083531e-02
   1.52304346e-02   1.15467509e-01   1.23706552e-02   6.18865858e-02
   5.32308045e-03   5.20776710e-03   2.15360122e-02   3.95591405e-03
   5.93811045e-01   4.06418188e-03   1.42505407e+01   4.27887410e-02
   1.09584754e

Iteration: 0
Alpha: [  1.95139141e-01   3.34041975e-02   2.21017905e-02   3.43669510e-02
   1.24384996e-02   1.41943913e-01   1.99241661e-02   1.22928210e-02
   4.70092874e-02   5.18177585e-03   1.37152948e-02   7.07585619e-03
   4.85065456e-03   2.04121377e-03   2.68225635e-02   5.19488330e-03
   1.17735893e-02   2.81076255e-03   3.15351950e-02   6.38361022e-03
   3.50352770e-01   9.31255796e-03   8.91711763e-03   3.87808527e-03
   2.96153993e+00   2.45563127e-03   1.37849694e-02   1.77973096e+02
   6.90339296e-01   8.10963708e-02   1.02232296e-01   5.06835942e-02
   5.28353389e-03   6.37314687e-03   7.78506854e-03   3.35466957e+01
   5.74626151e-02   1.11709002e-01   6.69594428e-03   6.50427867e-03
   1.29119296e-03   6.91400922e-03   2.04696263e-03   2.56483168e-02
   2.04221942e-03   3.75850517e-03   6.17108319e-03   1.97587570e-03
   5.57776142e-03   4.21968163e-03   2.42626734e-03   1.90001651e-02
   5.54376919e-02   2.34531800e-02   2.99844877e-02   2.05616019e-03
   3.62115084e

done in 15.539s
[ 1.0002  1.0002  1.0002  1.0002  1.0002]
RVC(alpha=0.0001, beta=0.0001, beta_fixed=False, bias_used=True, coef0=0.0,
  coef1=None, degree=3, kernel='rbf', n_iter=1, n_iter_posterior=50,
  threshold_alpha=1000000000.0, tol=0.001, verbose=True)
[         One    Two  Three   Four   Five
One     0.96  0.011  0.022  0.005   0.01
Two    0.014   0.91  0.006   0.09  0.005
Three  0.005  0.005  0.933  0.009   0.03
Four   0.009   0.07  0.011  0.872  0.025
Five   0.013  0.004  0.028  0.025   0.93,          One    Two  Three   Four   Five
One    0.952  0.008  0.008  0.001  0.006
Two    0.008  0.883  0.008  0.096    0.0
Three  0.013  0.003  0.944   0.01  0.027
Four   0.016  0.104  0.014  0.863  0.017
Five   0.011  0.003  0.026   0.03   0.95,          One    Two  Three   Four   Five
One     0.95  0.005  0.009  0.001  0.004
Two    0.011  0.883  0.002  0.094  0.001
Three  0.018  0.006  0.944  0.009  0.029
Four   0.011  0.106   0.01  0.859  0.029
Five   0.011    0.0  0.036  0.037  0.937

In [163]:
print EstParameters1[0].verbose

True
